In [1]:
!pip install -U --pre tensorflow=="2.2.0"

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import tensorflow as tf
print("Using tensorflow version: " + tf.__version__)
print("Using eager execution: " + str(tf.executing_eagerly())) 

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 3.0MB 32kB/s 
     |████████████████████████████████| 460kB 47.1MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
Using tensorflow version: 2.2.0
Using eager execution: True


In [2]:
import pathlib
from PIL import Image
import os
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import time
import random
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import io
import imageio

In [3]:
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2234, done.
remote: Counting objects: 100% (2234/2234), done.
remote: Compressing objects: 100% (1930/1930), done.
remote: Total 2234 (delta 541), reused 957 (delta 279), pack-reused 0
Receiving objects: 100% (2234/2234), 30.48 MiB | 27.14 MiB/s, done.
Resolving deltas: 100% (541/541), done.


In [4]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1580433 sha256=057b83bb0356042450446ea01a7b88255cd4498775dabe6a8640b9e26b28dee4
  Stored in directory: /tmp/pip-ephem-wheel-cache-jgi15x6y/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=7ca461601fbfe5f79429fbe384ec82d1adeb5df7c7b832c9f66e9837eafce840
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=d76da6b3c2dd65665218098feab41b3ed7e14a0cc84899debccdb9eaa5430ad3
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=dc077fe0ddd6c4790cc1a0b35c9c955fd908c8ee1be26d8e09be

ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.24.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1; python_version >= "3.0", but you'll have avro-python3 1.10.0 which is incompatible.


In [5]:
!pip install -U --pre tensorflow=="2.2.0"
import tensorflow as tf
print("Using tensorflow version: " + tf.__version__)
print("Using eager execution: " + str(tf.executing_eagerly())) 

  Using cached https://files.pythonhosted.org/packages/3d/be/679ce5254a8c8d07470efb4a4c00345fae91f766e64f1c2aece8796d7218/tensorflow-2.2.0-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a4/f5/926ae53d6a226ec0fda5208e0e581cffed895ccc89e36ba76a8e60895b78/tensorflow_estimator-2.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1d/74/0a6fcb206dcc72a6da9a62dd81784bfdbff5fedb099982861dc2219014fb/tensorboard-2.2.2-py3-none-any.whl
ERROR: tf-models-official 2.3.0 has requirement tensorflow>=2.3.0, but you'll have tensorflow 2.2.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.1
    Uninstalling tensorflow-2.3.

Using tensorflow version: 2.2.0
Using eager execution: True


In [6]:
import tensorflow as tf

from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import visualization_utils as viz_utils

In [7]:
from google.colab import drive
drive.mount("/content/drive")
!ls


Mounted at /content/drive
drive  models  sample_data


In [8]:
DATASET_PATH = "/content/drive/My Drive/Proiecte/MaskDetection/dataset"

masked_faces_paths = list(pathlib.Path(DATASET_PATH + "/masked_faces").glob('*'))
normal_faces_paths = list(pathlib.Path(DATASET_PATH + "/normal_faces").glob('*'))

length = len(masked_faces_paths)
print(f"Total number of samples: {length}")

split_ratio = 0.8
training_size = int(split_ratio * length)
validation_size = length - training_size
print(f"Dataset training size: {training_size} | Dataset validation size: {validation_size}")

training_paths = masked_faces_paths[:training_size] + normal_faces_paths[:training_size]
validation_paths = masked_faces_paths[training_size:] + normal_faces_paths[training_size:]

targets = None
with open(DATASET_PATH + "/targets.json") as json_file:
	targets = json.load(json_file)

Total number of samples: 680
Dataset training size: 544 | Dataset validation size: 136


In [9]:
image_path = training_paths[2]
image_name = str(image_path).split(os.sep)[8]
str(image_path).split(os.sep)[8]
targets[image_name]
image = Image.open(image_path).convert("RGB")
np.array(targets[image_name]["bbox"]).shape
plt.imshow(image)


'101.jpg'

{'bbox': [[194, 246, 656, 708]]}

(1, 4)

In [10]:
def get_dataset():
    print('Prepping data.')
    num_classes = 2
    label_id_offset = 1

    train_image_tensors = []
    gt_classes_one_hot_tensors = []
    gt_box_tensors = []

    for idx, image_path in enumerate(training_paths):
        print(image_path)
        masked = True if idx < training_size else False
        train_image_np = np.array(Image.open(image_path).convert("RGB"))
        image_name = str(image_path).split(os.sep)[8]
        gt_box_np = np.array(targets[image_name]["bbox"])

        train_image_tensors.append(
            tf.expand_dims(
                tf.convert_to_tensor(
                    train_image_np, dtype=tf.float32
                ), axis=0)
            )

        gt_box_tensors.append(tf.convert_to_tensor(gt_box_np, dtype=tf.float32))
        zero_indexed_groundtruth_classes = tf.convert_to_tensor(
            np.zeros(shape=[gt_box_np.shape[0]], dtype=np.int32) if masked else np.ones(shape=[gt_box_np.shape[0]], dtype=np.int32)
        )
        
        gt_classes_one_hot_tensors.append(
            tf.one_hot(
                zero_indexed_groundtruth_classes, num_classes
            )
        )
    print('Done prepping data.')
    return train_image_tensors, gt_classes_one_hot_tensors, gt_box_tensors

In [11]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

--2020-10-09 21:08:36--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.128, 2607:f8b0:400e:c03::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M   172MB/s    in 1.4s    

2020-10-09 21:08:37 (172 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]



In [12]:
def get_model():
    print('Building model and restoring weights for fine-tuning...', flush=True)
    num_classes = 2
    pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
    # pipeline_config = 'models/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
    # pipeline_config = 'models/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config'

    # pipeline_config = 'models/research/object_detection/test_data/pipeline.config'
    # checkpoint_path = 'models/research/object_detection/test_data/checkpoint_mobilenet/ckpt-0'
    checkpoint_path = 'models/research/object_detection/test_data/checkpoint/ckpt-0'
    # checkpoint_path = 'models/research/object_detection/test_data/checkpoint_mobilenet_fpn/ckpt-0'

    # print(tf.train.list_variables(tf.train.latest_checkpoint("models/research/object_detection/test_data/checkpoint_mobilenet_fpn")))

    configs = config_util.get_configs_from_pipeline_file(pipeline_config)
    model_config = configs['model']
    model_config.ssd.num_classes = num_classes
    model_config.ssd.freeze_batchnorm = True
    detection_model = model_builder.build(model_config=model_config, is_training=True)

    fake_box_predictor = tf.compat.v2.train.Checkpoint(
        _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
        _box_prediction_head=detection_model._box_predictor._box_prediction_head
    )

    fake_model = tf.compat.v2.train.Checkpoint(
        _feature_extractor=detection_model._feature_extractor,
        _box_predictor=fake_box_predictor
    )
    ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
    ckpt.restore(checkpoint_path).expect_partial()

    image, shapes = detection_model.preprocess(tf.zeros([1, 320, 320, 3]))
    prediction_dict = detection_model.predict(image, shapes)
    _ = detection_model.postprocess(prediction_dict, shapes)
    print('Weights restored!')

    return detection_model

In [13]:
def train_step(
    model,
    image_tensors,
    groundtruth_boxes_list,
    groundtruth_classes_list,
    vars_to_fine_tune,
    optimizer
):
    batch_size = 4

    shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
    model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list
    )

    with tf.GradientTape() as tape:
        preprocessed_images = tf.concat(
            [model.preprocess(image_tensor)[0]
            for image_tensor in image_tensors], axis=0
        
        )
        prediction_dict = model.predict(preprocessed_images, shapes)
        losses_dict = model.loss(prediction_dict, shapes)
        total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
        gradients = tape.gradient(total_loss, vars_to_fine_tune)
        optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
    
    return total_loss

In [14]:
def train_loop(config):
    epochs = config["epochs"]
    learning_rate = config["learning_rate"]
    clip_gradients_value = config["clip_gradients_value"] if "clip_gradients_value" in config else None
    evaluation_interval = config["evaluation_interval"] if "evaluation_interval" in  config else 1
    metrics_interval = config["metrics_interval"] if "metrics_interval" in config else 1

    detection_model = get_model()
    train_image_tensors, gt_classes_one_hot_tensors, gt_box_tensors = get_dataset()

    trainable_variables = detection_model.trainable_variables
    to_fine_tune = []
    prefixes_to_train = [
        'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
        'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead'
    ]
    
    for var in trainable_variables:
        if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
            to_fine_tune.append(var)

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

    train_losses = []
    epoch_loss_avg = tf.keras.metrics.Mean()

    print("Start training!")
    for epoch in range(epochs):
        epoch_start_time = time.time()

        for idx in range(100):
            all_keys = list(range(len(training_paths)))
            random.shuffle(all_keys)
            example_keys = all_keys[:4]

            gt_boxes_list = [gt_box_tensors[key] for key in example_keys]
            gt_classes_list = [gt_classes_one_hot_tensors[key] for key in example_keys]
            image_tensors = [train_image_tensors[key] for key in example_keys]

            total_loss = train_step(detection_model, image_tensors, gt_boxes_list, gt_classes_list, to_fine_tune, optimizer)

        # for x, y in training_dataset:
        #     total_loss = train_step(
        #         to_fine_tune
        #     )

            epoch_loss_avg(total_loss)

        train_losses.append(epoch_loss_avg.result())
        epoch_time_elapsed = time.time() - epoch_start_time

        if epoch % metrics_interval == 0:
            print(
                "Epoch {:03d} | Loss: {:.3f} | Time: {:.0f}m {:.0f}s"
                .format(epoch, epoch_loss_avg.result(), epoch_time_elapsed // 60, epoch_time_elapsed % 60)
            )

        # if epoch % evaluation_interval == 0:
        #     evaluate_model(model, validation_dataset)

        epoch_loss_avg.reset_states()
    print("Training finished")

    return detection_model

In [15]:
config = {
    "epochs": 100,
    "learning_rate": 0.001
}

trained_model = train_loop(config)

Building model and restoring weights for fine-tuning...
Weights restored!
Prepping data.
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/10.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/100.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/101.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/102.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/104.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/105.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/106.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/107.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/108.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/109.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/11.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/111.jpg
/

In [18]:
output_directory = 'output/'
output_checkpoint_dir = os.path.join(output_directory, 'checkpoint')

# To save checkpoint for TFLite conversion.
exported_ckpt = tf.compat.v2.train.Checkpoint(model=trained_model)
ckpt_manager = tf.train.CheckpointManager(exported_ckpt, output_checkpoint_dir, max_to_keep=1)
ckpt_manager.save()
print('Checkpoint saved!')

'output/checkpoint/ckpt-1'

In [20]:
pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
pipeline_proto = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_proto, output_directory)

INFO:tensorflow:Writing pipeline config file to output/pipeline.config


In [22]:
%%bash
python models/research/object_detection/export_tflite_graph_tf2.py \
  --pipeline_config_path output/pipeline.config \
  --trained_checkpoint_dir output/checkpoint \
  --output_directory tflite

2020-10-09 23:17:46.964665: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-10-09 23:17:46.969281: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-09 23:17:46.969652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-09 23:17:46.969960: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-10-09 23:17:46.971499: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-10-09 23:17:46.973213: I tensorflow/stream_executor/platform/default/d

In [ ]:
!tflite_convert --saved_model_dir=tflite/saved_model --output_file=tflite/model.tflite

In [26]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  print(path)
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [24]:
def detect(detection_model, input_tensor):
  """Run detection on an input image.

  Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

  Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
      and `detection_scores`).
  """
  preprocessed_image, shapes = detection_model.preprocess(input_tensor)
  prediction_dict = detection_model.predict(preprocessed_image, shapes)
  return detection_model.postprocess(prediction_dict, shapes)

In [31]:
test_images_np = []

category_index = {1: {'id': 1, 'name': 'rubber_ducky'}, 2: {'id': 2, 'name': 'rubber_ducky_2'}}

for idx, image_path in enumerate(training_paths[:5]):
    test_images_np.append(
        np.expand_dims(
            load_image_into_numpy_array(str(image_path)), axis=0
        )
    )

label_id_offset = 2
for i in range(len(test_images_np)):
    input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
    detections = detect(trained_model, input_tensor)
    print(detections)
    plot_detections(
        test_images_np[i][0],
        detections['detection_boxes'][0].numpy(),
        detections['detection_classes'][0].numpy().astype(np.uint32) + label_id_offset,
        detections['detection_scores'][0].numpy(),
        category_index, figsize=(15, 20), 
        # image_name="gif_frame_" + ('%02d' % i) + ".jpg"
    )

/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/10.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/100.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/101.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/102.jpg
/content/drive/My Drive/Proiecte/MaskDetection/dataset/masked_faces/104.jpg
{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[0.6852752 , 0.        , 1.        , 0.31614536],
        [0.74453294, 0.        , 1.        , 0.44149196],
        [0.6822897 , 0.00489178, 1.        , 0.51796925],
        [0.2963345 , 0.        , 1.        , 0.69228965],
        [0.62479943, 0.        , 1.        , 0.18705826],
        [0.8197371 , 0.63946795, 1.        , 1.        ],
        [0.        , 0.        , 1.        , 0.37954092],
        [0.7476131 , 0.5012815 , 1.        , 1.        ],
        [0.8197371 , 0.63946795, 1.        , 1.        ],
        [0.3798867 , 0.071